In [3]:
import pandas as pd
import numpy as np

In [25]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [35]:
reserve_tb=pd.read_csv('./data/reserve.csv',encoding='UTF-8')
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


--------
### hotel_id를 키 값으로 하여 고객수와 예약수를 집계해보자 (Not Awesome)
* groupby(), size(), unique(), merge()

In [8]:
# 현재 행 번호로 고친다.
rsv_cnt_tb=reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.head()

,hotel_id,0
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [10]:
# 열 이름 변경
rsv_cnt_tb.columns=['hotel_id','rsv_cnt']
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [14]:
# hotel_id를 집약 단위로 설정하고, customer_id값을 nunique함수를 이용하여 고객 수 카운팅
cus_cnt_tb=reserve_tb.groupby('hotel_id')['customer_id'].nunique().reset_index()
cus_cnt_tb.head()

,hotel_id,customer_id
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [15]:
# 열 이름 변경
cus_cnt_tb.columns=['hotel_id','cus_cnt']
cus_cnt_tb.head()

,hotel_id,cus_cnt
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [17]:
# merge함수를 이용해 hotel_id를 결합 키로 결합
pd.merge(rsv_cnt_tb, cus_cnt_tb, on='hotel_id').head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


### hotel_id를 키 값으로 하여 고객수와 예약수를 집계해보자 (Awesome)
* groupby(), agg(), reset_index()

In [34]:
# rserve_id를 대상으로 count 함수를 적용
# customer_id를 대상으로 ununique함수를 적용
result=reserve_tb.groupby('hotel_id').agg({'reserve_id':'count', 'customer_id':'nunique'})

# reset_index함수로 열 번호를 다시 설정한다.
# inplace=True이므로 컬럼명 다시 갱신
result.reset_index(inplace=True)
result.columnns=['hotel_id','rsv_cnt','cus_cnt']
result.head()

,hotel_id,reserve_id,customer_id
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
